# SDM Componentes
Automatización de obtención de datos de [SDM Componentes](https://docs.google.com/spreadsheets/d/1Dene2gQPNl7ZXIKaf4AYqKzeOahMJ2uA-Vity1eXPUo/edit?gid=1637837061#gid=1637837061).  
El resultado se guardará en `./sdm_componentes/SDM_Componentes_final.csv`.  

1) Descargar como csv desde https://docs.google.com/spreadsheets/d/1aLELrHoXPwm8CSY2a6DN6DRqoIkruHQO37JrQcaZFNM/edit?gid=855523154#gid=855523154 y guardar como `./sdm_componentes/sdm_ultimos_confs_tabla.csv`
2) Descargar como csv desde https://docs.google.com/spreadsheets/d/1aLELrHoXPwm8CSY2a6DN6DRqoIkruHQO37JrQcaZFNM/edit?gid=75156217#gid=75156217 y guardar como `./sdm_componentes/sdm_parametros_confs.csv`

Actualizamos con resultado (`./sdm_componentes/SDM_Componentes_final.csv`):  
https://docs.google.com/spreadsheets/d/1aLELrHoXPwm8CSY2a6DN6DRqoIkruHQO37JrQcaZFNM/edit?gid=1324569054#gid=1324569054


In [1]:
import pandas as pd
import numpy as np

import warnings
warnings.filterwarnings("ignore")
pd.set_option('display.max_columns', None)

Q_actual = ['3Q25'] # <----------------------- ATENTOS CON ESTO, QUE SINO DETECTA QUE NO HAY CAMBIOS

Lectura

In [2]:
repos = pd.read_csv('repos/repos.csv')

sdm_confs       = pd.read_csv('./sdm_componentes/sdm_ultimos_confs_tabla.csv',skiprows=5).dropna(subset='tabla')
sdm_confs       .drop(columns=sdm_confs.columns[sdm_confs.columns.str.startswith('Unnamed')],inplace=True)
sdm_version     = sdm_confs[["Q","version.1"]].dropna().rename(columns={"version.1":'version'})
sdm_confs       = sdm_confs.drop(columns='Q').merge(sdm_version,how='left',on='version')

sdm_fecha       = sdm_confs[["Fecha2text","Version2"]].dropna().rename(columns={"Version2":'version', "Fecha2text":'Fecha'})
sdm_fecha.Fecha = pd.to_datetime(sdm_fecha.Fecha, format= '%d/%m/%Y')
sdm_fecha       = sdm_fecha.sort_values('Fecha',ascending=False).drop_duplicates('version')
sdm_fecha       .Fecha = sdm_fecha.Fecha.dt.strftime('%d/%m/%Y')
sdm_confs       = sdm_confs.drop(columns='Fecha').merge(sdm_fecha,how='left',on='version')


sdm_params      = pd.read_csv('./sdm_componentes/sdm_parametros_confs.csv').dropna(subset='conf')
sdm_params      .drop(columns=sdm_params.columns[sdm_params.columns.str.startswith('Unnamed')],inplace=True)

sdm_unique_params = sdm_params[['unique parameters.1','params.1']].dropna(subset='unique parameters.1')

In [3]:
conf_q = sdm_confs.groupby(['conf','Q']).count()['tabla'].reset_index().pivot(index='conf', columns='Q', values='tabla').fillna(0)>=1
Q_no_actual = conf_q.columns.values.tolist()
Q_no_actual.remove(Q_actual[0])
conf_q['Tipo'] = np.where(conf_q[Q_actual[0]] == False,'Sin cambios', 
                          np.where( (conf_q[Q_no_actual].any(axis=1) == True), 'Modificado', 'Nuevo'))
conf_q = conf_q['Tipo']
conf_q = conf_q.reset_index()
sdm_confs = sdm_confs.merge(conf_q, on='conf', how='left')

## Transformación

### 1) Params

In [4]:
#rellenar en origen cuando falten!!!!
sdm_params_clean        = sdm_params[sdm_params["STATUS IMPLANTATION"] == "DELIVERED"]
sdm_params_clean        = sdm_params_clean[['conf','unique parameters','params']].drop_duplicates(subset=['conf','params'])
sdm_params_clean.params = sdm_params_clean.params.str.replace('^,\ ','',regex=True)
sdm_params_clean        = sdm_params_clean.sort_values('params',ascending=True)
print('---------------\n- Null params -\n---------------\n ¡REVISAR! (Rellenar cuando falten en origen):\n----------------------------------------------')
print('https://docs.google.com/spreadsheets/d/1VlE-2bK8370eYNGi5kEeXWonZ_emMrw8VAS-fNiwol8/edit?gid=1150342608#gid=1150342608&range=K:L')
sdm_params_clean[sdm_params_clean.params.isnull()]
########################################################

---------------
- Null params -
---------------
 ¡REVISAR! (Rellenar cuando falten en origen):
----------------------------------------------
https://docs.google.com/spreadsheets/d/1VlE-2bK8370eYNGi5kEeXWonZ_emMrw8VAS-fNiwol8/edit?gid=1150342608#gid=1150342608&range=K:L


,conf,unique parameters,params


In [5]:
# sdm_params_clean.params.str.split()
sdm_params_clean_split = (sdm_params_clean.set_index('conf')
            ['params'].str.split(',\s*', expand=True)
            .stack()
            .reset_index(name='param')
            .drop(columns='level_1')
            )
sdm_params_clean_split.param.unique()

array(['CATALOG_COLUMN_0', 'CATALOG_CODE_0', 'CATALOG_TABLE_0', 'SOURCE',
       'CATALOG_COLUMN_1', 'CATALOG_CODE_1', 'CATALOG_TABLE_1',
       'extralibs', 'CATALOG_COLUMN_10', 'CATALOG_CODE_10',
       'CATALOG_TABLE_10', 'CATALOG_COLUMN_11', 'CATALOG_CODE_11',
       'CATALOG_TABLE_11', 'CATALOG_COLUMN_2', 'CATALOG_CODE_2',
       'CATALOG_TABLE_2', 'CATALOG_COLUMN_3', 'CATALOG_CODE_3',
       'CATALOG_TABLE_3', 'CATALOG_COLUMN_4', 'CATALOG_CODE_4',
       'CATALOG_TABLE_4', 'CATALOG_COLUMN_5', 'CATALOG_CODE_5',
       'CATALOG_TABLE_5', 'CATALOG_COLUMN_6', 'CATALOG_CODE_6',
       'CATALOG_TABLE_6', 'CATALOG_COLUMN_7', 'CATALOG_CODE_7',
       'CATALOG_TABLE_7', 'CATALOG_COLUMN_8', 'CATALOG_CODE_8',
       'CATALOG_TABLE_8', 'CATALOG_COLUMN_9', 'CATALOG_CODE_9',
       'CATALOG_TABLE_9', 'CLOSING_DATE', 'ADOBE_REPORT_SUITE_NAME',
       'DATE_VOLUMETRIC', 'LOWER_BOUND', 'UPPER_BOUND',
       'FECHA_VOLUMETRIC', 'CUTOFF_DATE', 'IS_VOL_CRITICAL',
       'OPERATION_DATE', 'PREVIOUS_C

In [6]:
sdm_params_clean_params = sdm_params_clean_split.groupby('conf')['param'].agg(params_clean= lambda x: ', '.join(sorted(list(set(x))))).reset_index()
# print(sdm_params_clean_params.params_clean.unique())
sdm_params_clean_params

,conf,params_clean
0,B0HN_agg-output-ems-hgl_KSAG_holding-business-...,CUTOFF_DATE
1,B0HN_agg-output-ems-hgl_MRField.conf,CUTOFF_DATE
2,B0HN_agg-output-ems-hgl_MRObject.conf,"CUTOFF_DATE, FECHA_VOLUMETRIC, LOWER_BOUND, UP..."
3,B0HN_sust-group-mark_KSAG_holding-business-gro...,"CUTOFF_DATE, SOURCE"
4,B0HN_sust-group-mark_MRField.conf,"CUTOFF_DATE, SOURCE"
...,...,...
838,YFVH_cards-calc-balances_KCOG_currency_MMRule....,CUTOFF_DATE
839,YFVH_cards-calc-balances_KCOG_entity_MMRule.conf,CUTOFF_DATE
840,YFVH_cards-calc-balances_KSAN_contract-global-...,CUTOFF_DATE
841,YFVH_cards-calc-balances_MRField.conf,CUTOFF_DATE


### 2) SDM Confs

In [7]:
dict_conf_type = {
    "MRField":  "1",
    "MRObject": "2",
    "MMRule":   "3"
}
dict_periodicidad = {
    'Daily':    'Diaria',
    'Monthly':  'Mensual',
    'Annually': 'Anual',
    'Punctual': 'Puntual',
    'Weekly':   'Semanal'
}

sdm_confs                               = sdm_confs[sdm_confs["STATUS IMPLANTATION"] == "DELIVERED"]
sdm_confs                               = sdm_confs.merge(sdm_params_clean_params, how='left', on='conf')
sdm_confs['Version_ord']                = pd.to_numeric(sdm_confs.version.str.replace('.','',regex=False))
sdm_confs                               .sort_values('Version_ord',ascending=False,inplace=True)
sdm_confs                               .drop_duplicates(subset='conf',keep='first',inplace=True)

sdm_confs['conf_type']                  = sdm_confs.conf.str.extract('((_MR|_MM).*\.conf)')[0].str.replace('_','').str.replace('.conf','')
sdm_confs['conf_ord']                   = sdm_confs['conf_type'] .replace(dict_conf_type)
sdm_confs                               .sort_values(by=['tabla','conf_type'],inplace=True)
sdm_confs['Fecha / Release']            = sdm_confs["Fecha"]
sdm_confs['Tipo']                       = sdm_confs['Tipo']
sdm_confs['UUAA']                       = sdm_confs.tabla.str[:6].str[2:].str.upper()
sdm_confs['Proceso / Tabla Master']     = sdm_confs.tabla
sdm_confs['Tabla']                      = sdm_confs.tabla
sdm_confs['Versión']                    = 'latest'
sdm_confs['Etapa / Fase']               = 'Master'
sdm_confs['Tipo Componente']            = "Hammurabi"
sdm_confs['Subtipo Componente']         = sdm_confs['conf_type']
sdm_confs['Versión SW']                 = '"' + sdm_confs.version.astype(str) + '"'

sdm_confs['job'] = sdm_confs['job europa'].str.split('europa/').str[1].str.split('.json?').str[0]
sdm_confs['job_europa_hiper']           = "=HYPERLINK(\"" + sdm_confs['job europa'] + "\";\"" + sdm_confs['job'] + "\")"
sdm_confs['job_america_ada_hiper']      = "=HYPERLINK(\"" + sdm_confs['job america'].str.replace('/america/','/america-ada/') + "\";\"" + sdm_confs['job'] + "\")"
sdm_confs['job_america_hiper']          = "=HYPERLINK(\"" + sdm_confs['job america'] + "\";\"" + sdm_confs['job'] + "\")"
sdm_confs['Job ADA Europa (Bitbucket)']     = sdm_confs['job_europa_hiper']
sdm_confs['Job ADA América (Bitbucket)']    = sdm_confs['job_america_ada_hiper']
sdm_confs['Job Datio América (Bitbucket)']  = sdm_confs['job_america_hiper']
sdm_confs['Conf (opcional)']            = "=HYPERLINK(\"" + sdm_confs['url conf'] + "\";\"" + sdm_confs['conf'] + "\")" 
sdm_confs['Variables Job']              = sdm_confs['params_clean']
sdm_confs['Ejemplo (opcional)']         = ' '
sdm_confs['ID Orden']                   = ' '
sdm_confs['Periodicidad']               = sdm_confs['periodicidad'].str.strip().replace(dict_periodicidad)
sdm_confs['Mixta']                      = np.where(sdm_confs['Comentarios'] == 'Holding', 'Holding to Local', sdm_confs['Mixta'])
sdm_confs['Comentarios']                = sdm_confs.Comentarios
sdm_confs['ESP']                        = np.where(sdm_confs['Comentarios'] == 'Holding', False, True)
sdm_confs['MEX']                        = np.where(sdm_confs['Comentarios'] == 'Holding', False, True)
sdm_confs['PER']                        = np.where(sdm_confs['Comentarios'] == 'Holding', False, True)
sdm_confs['COL']                        = np.where(sdm_confs['Comentarios'] == 'Holding', False, True)
sdm_confs['ARG']                        = np.where(sdm_confs['Comentarios'] == 'Holding', False, True)

sdm_confs = sdm_confs.merge(repos,how='left',on='UUAA')
print("proyectos nulos:", sdm_confs.project.isnull().sum())
if sdm_confs.project.isnull().sum() > 0:
    raise print("revisar proyectos en repos/repos.csv")
#seguimos apuntando a Bitbucket
#sdm_confs['Conf (opcional)']            = 'https://bitbucket.globaldevtools.bbva.com/bitbucket/projects/' + sdm_confs['project'] + '/repos/glgldh360gloqlt/browse/hammurabi/confs/master/' + sdm_confs['UUAA'].str.lower() + '/' + sdm_confs['conf']
sdm_confs.head()

proyectos nulos: 0


,SCHEMA,job europa,job america,job url,tabla,conf,STATUS IMPLANTATION,version,uuaa,url conf,Variables Job,count param,periodicidad,Mixta,Comentarios,conf_bitbucket,version.1,conf.1,job url.1,version.2,Version2,Fecha2,Fecha2text,Q,Fecha,Tipo,params_clean,Version_ord,conf_type,conf_ord,Fecha / Release,UUAA,Proceso / Tabla Master,Tabla,Versión,Etapa / Fase,Tipo Componente,Subtipo Componente,Versión SW,job,job_europa_hiper,job_america_ada_hiper,job_america_hiper,Job ADA Europa (Bitbucket),Job ADA América (Bitbucket),Job Datio América (Bitbucket),Conf (opcional),Ejemplo (opcional),ID Orden,Periodicidad,ESP,MEX,PER,COL,ARG,project
0,https://artifactory.globaldevtools.bbva.com/ui...,https://bitbucket.globaldevtools.bbva.com/bitb...,https://bitbucket.globaldevtools.bbva.com/bitb...,https://bitbucket.globaldevtools.bbva.com/bitb...,t_b0hn_agg_output_ems_hgl,B0HN_agg-output-ems-hgl_KSAG_holding-business-...,DELIVERED,15.1.0,B0HN,https://artifactory.globaldevtools.bbva.com/ui...,CUTOFF_DATE,1.0,Monthly,Holding to Local,Holding,NaN,17.4.0,B0HN_agg-output-ems-hgl_KSAG_holding-business-...,https://bitbucket.globaldevtools.bbva.com/bitb...,NaN,17.2.0,45868,30/07/2025,4Q21,06/11/2024,Sin cambios,CUTOFF_DATE,1510,MMRule,3,06/11/2024,B0HN,t_b0hn_agg_output_ems_hgl,t_b0hn_agg_output_ems_hgl,latest,Master,Hammurabi,MMRule,"""15.1.0""",b0hn-gl-hmm-qlt-aggoutputemshglmmrule-01,"=HYPERLINK(""https://bitbucket.globaldevtools.b...","=HYPERLINK(""https://bitbucket.globaldevtools.b...","=HYPERLINK(""https://bitbucket.globaldevtools.b...","=HYPERLINK(""https://bitbucket.globaldevtools.b...","=HYPERLINK(""https://bitbucket.globaldevtools.b...","=HYPERLINK(""https://bitbucket.globaldevtools.b...","=HYPERLINK(""https://artifactory.globaldevtools...",,,Mensual,False,False,False,False,False,GL_B0HN_APP-ID-23103_DSG
1,https://artifactory.globaldevtools.bbva.com/ui...,https://bitbucket.globaldevtools.bbva.com/bitb...,https://bitbucket.globaldevtools.bbva.com/bitb...,https://bitbucket.globaldevtools.bbva.com/bitb...,t_b0hn_agg_output_ems_hgl,B0HN_agg-output-ems-hgl_MRField.conf,DELIVERED,15.1.0,B0HN,https://artifactory.globaldevtools.bbva.com/ui...,CUTOFF_DATE,1.0,Monthly,Holding to Local,Holding,NaN,17.5.0,B0HN_agg-output-ems-hgl_MRField.conf,https://bitbucket.globaldevtools.bbva.com/bitb...,NaN,17.5.0,25/08/2025,14/08/2025,1Q21,06/11/2024,Sin cambios,CUTOFF_DATE,1510,MRField,1,06/11/2024,B0HN,t_b0hn_agg_output_ems_hgl,t_b0hn_agg_output_ems_hgl,latest,Master,Hammurabi,MRField,"""15.1.0""",b0hn-gl-hmm-qlt-aggoutputemshglmrfield-01,"=HYPERLINK(""https://bitbucket.globaldevtools.b...","=HYPERLINK(""https://bitbucket.globaldevtools.b...","=HYPERLINK(""https://bitbucket.globaldevtools.b...","=HYPERLINK(""https://bitbucket.globaldevtools.b...","=HYPERLINK(""https://bitbucket.globaldevtools.b...","=HYPERLINK(""https://bitbucket.globaldevtools.b...","=HYPERLINK(""https://artifactory.globaldevtools...",,,Mensual,False,False,False,False,False,GL_B0HN_APP-ID-23103_DSG
2,https://artifactory.globaldevtools.bbva.com/ui...,https://bitbucket.globaldevtools.bbva.com/bitb...,https://bitbucket.globaldevtools.bbva.com/bitb...,https://bitbucket.globaldevtools.bbva.com/bitb...,t_b0hn_agg_output_ems_hgl,B0HN_agg-output-ems-hgl_MRObject.conf,DELIVERED,15.1.0,B0HN,https://artifactory.globaldevtools.bbva.com/ui...,"CUTOFF_DATE, FECHA_VOLUMETRIC, LOWER_BOUND, UP...",1.0,Monthly,Holding to Local,Holding,NaN,17.4.0,B0HN_agg-output-ems-hgl_MRObject.conf,https://bitbucket.globaldevtools.bbva.com/bitb...,NaN,17.4.0,45883,30/07/2025,3Q24,06/11/2024,Sin cambios,"CUTOFF_DATE, FECHA_VOLUMETRIC, LOWER_BOUND, UP...",1510,MRObject,2,06/11/2024,B0HN,t_b0hn_agg_output_ems_hgl,t_b0hn_agg_output_ems_hgl,latest,Master,Hammurabi,MRObject,"""15.1.0""",b0hn-gl-hmm-qlt-aggoutputemshglmrobject-01,"=HYPERLINK(""https://bitbucket.globaldevtools.b...","=HYPERLINK(""https://bitbucket.globaldevtools.b...","=HYPERLINK(""https://bitbucket.globaldevtools.b...","=HYPERLINK(""https://bitbucket.globaldevtools.b...","=HY

### 3) SDM Componentes - Final

In [8]:
sdm_componentes         = sdm_confs[[
    'Fecha / Release','Tipo','UUAA','Proceso / Tabla Master','Tabla','Versión','Etapa / Fase','Tipo Componente','Subtipo Componente','Versión SW',
    'Job ADA Europa (Bitbucket)','Job ADA América (Bitbucket)','Job Datio América (Bitbucket)','Conf (opcional)','Variables Job','Ejemplo (opcional)', 'ID Orden', 'Periodicidad', 'Mixta', 'Comentarios','ESP', 'MEX', 'PER', 'COL', 'ARG'
    ]]

sdm_componentes         .Periodicidad.fillna('Diaria',inplace=True)
print('NULOS:')
print(sdm_componentes.isnull().sum())
sdm_componentes['type'] = sdm_componentes['Conf (opcional)'].str.extract(r'_(.{6,8}).conf').replace({'MRField':1, 'MRObject':2, 'MMRule':3})
sdm_componentes         .sort_values(['Tabla','type'], inplace=True)
sdm_componentes         .drop(columns='type', inplace=True)
sdm_componentes         .to_csv('./sdm_componentes/SDM_Componentes_final.csv',sep=';',index=False)
sdm_componentes         .tail(2)

NULOS:
Fecha / Release                  0
Tipo                             0
UUAA                             0
Proceso / Tabla Master           0
Tabla                            0
Versión                          0
Etapa / Fase                     0
Tipo Componente                  0
Subtipo Componente               0
Versión SW                       0
Job ADA Europa (Bitbucket)       0
Job ADA América (Bitbucket)      0
Job Datio América (Bitbucket)    0
Conf (opcional)                  0
Variables Job                    0
Ejemplo (opcional)               0
ID Orden                         0
Periodicidad                     0
Mixta                            0
Comentarios                      0
ESP                              0
MEX                              0
PER                              0
COL                              0
ARG                              0
dtype: int64


,Fecha / Release,Tipo,UUAA,Proceso / Tabla Master,Tabla,Versión,Etapa / Fase,Tipo Componente,Subtipo Componente,Versión SW,Job ADA Europa (Bitbucket),Job ADA América (Bitbucket),Job Datio América (Bitbucket),Conf (opcional),Variables Job,Ejemplo (opcional),ID Orden,Periodicidad,Mixta,Comentarios,ESP,MEX,PER,COL,ARG
839,16/02/2024,Sin cambios,YFVH,t_yfvh_cards_calc_balances,t_yfvh_cards_calc_balances,latest,Master,Hammurabi,MMRule,"""13.0.3""","=HYPERLINK(""https://bitbucket.globaldevtools.b...","=HYPERLINK(""https://bitbucket.globaldevtools.b...","=HYPERLINK(""https://bitbucket.globaldevtools.b...","=HYPERLINK(""https://artifactory.globaldevtools...",CUTOFF_DATE,,,Diaria,DQ Local,Local,True,True,True,True,True
840,16/02/2024,Sin cambios,YFVH,t_yfvh_cards_calc_balances,t_yfvh_cards_calc_balances,latest,Master,Hammurabi,MMRule,"""13.0.3""","=HYPERLINK(""https://bitbucket.globaldevtools.b...","=HYPERLINK(""https://bitbucket.globaldevtools.b...","=HYPERLINK(""https://bitbucket.globaldevtools.b...","=HYPERLINK(""https://artifactory.globaldevtools...",CUTOFF_DATE,,,Diaria,DQ Local,Local,True,True,True,True,True


In [9]:
sdm_componentes[sdm_componentes['Job ADA Europa (Bitbucket)'].isnull()]

,Fecha / Release,Tipo,UUAA,Proceso / Tabla Master,Tabla,Versión,Etapa / Fase,Tipo Componente,Subtipo Componente,Versión SW,Job ADA Europa (Bitbucket),Job ADA América (Bitbucket),Job Datio América (Bitbucket),Conf (opcional),Variables Job,Ejemplo (opcional),ID Orden,Periodicidad,Mixta,Comentarios,ESP,MEX,PER,COL,ARG


**Filtramos X tablas por orden**

In [10]:
tablas=[
    't_kctd_interaction_hit_quality',
    't_kceg_customer_engagement'
]
sdm_componentes_filtrado        = sdm_componentes[sdm_componentes.Tabla.isin(tablas)]
sdm_componentes_filtrado.Tabla  = sdm_componentes_filtrado.Tabla.astype("category").cat.set_categories(tablas)
sdm_componentes_filtrado        = sdm_componentes_filtrado.sort_values('Tabla')
sdm_componentes_filtrado        .to_csv('./sdm_componentes/filtrado/SDM_Componentes_final_FILTRADO.csv',sep=';',index=False)
sdm_componentes_filtrado


,Fecha / Release,Tipo,UUAA,Proceso / Tabla Master,Tabla,Versión,Etapa / Fase,Tipo Componente,Subtipo Componente,Versión SW,Job ADA Europa (Bitbucket),Job ADA América (Bitbucket),Job Datio América (Bitbucket),Conf (opcional),Variables Job,Ejemplo (opcional),ID Orden,Periodicidad,Mixta,Comentarios,ESP,MEX,PER,COL,ARG
332,06/11/2024,Sin cambios,KCTD,t_kctd_interaction_hit_quality,t_kctd_interaction_hit_quality,latest,Master,Hammurabi,MRField,"""15.1.0""","=HYPERLINK(""https://bitbucket.globaldevtools.b...","=HYPERLINK(""https://bitbucket.globaldevtools.b...","=HYPERLINK(""https://bitbucket.globaldevtools.b...","=HYPERLINK(""https://artifactory.globaldevtools...","ADOBE_REPORT_SUITE_NAME, CLOSING_DATE",,,Diaria,DQ Local,Both,True,True,True,True,True
333,06/11/2024,Sin cambios,KCTD,t_kctd_interaction_hit_quality,t_kctd_interaction_hit_quality,latest,Master,Hammurabi,MRObject,"""15.1.0""","=HYPERLINK(""https://bitbucket.globaldevtools.b...","=HYPERLINK(""https://bitbucket.globaldevtools.b...","=HYPERLINK(""https://bitbucket.globaldevtools.b...","=HYPERLINK(""https://artifactory.globaldevtools...","ADOBE_REPORT_SUITE_NAME, CLOSING_DATE, DATE_VO...",,,Diaria,DQ Local,Both,True,True,True,True,True
184,29/04/2025,Sin cambios,KCEG,t_kceg_customer_engagement,t_kceg_customer_engagement,latest,Master,Hammurabi,MRField,"""16.1.0""","=HYPERLINK(""https://bitbucket.globaldevtools.b...","=HYPERLINK(""https://bitbucket.globaldevtools.b...","=HYPERLINK(""https://bitbucket.globaldevtools.b...","=HYPERLINK(""https://artifactory.globaldevtools...",CUTOFF_DATE,,,Mensual,DQ Local,Local,True,True,True,True,True
185,06/11/2024,Sin cambios,KCEG,t_kceg_customer_engagement,t_kceg_customer_engagement,latest,Master,Hammurabi,MRObject,"""15.1.0""","=HYPERLINK(""https://bitbucket.globaldevtools.b...","=HYPERLINK(""https://bitbucket.globaldevtools.b...","=HYPERLINK(""https://bitbucket.globaldevtools.b...","=HYPERLINK(""https://artifactory.globaldevtools...","CUTOFF_DATE, FECHA_VOLUMETRIC, LOWER_BOUND, UP...",,,Mensual,DQ Local,Local,True,True,True,True,True
183,06/11/2024,Sin cambios,KCEG,t_kceg_customer_engagement,t_kceg_customer_engagement,latest,Master,Hammurabi,MMRule,"""15.1.0""","=HYPERLINK(""https://bitbucket.globaldevtools.b...","=HYPERLINK(""https://bitbucket.globaldevtools.b...","=HYPERLINK(""https://bitbucket.globaldevtools.b...","=HYPERLINK(""https://artifactory.globaldevtools...",CUTOFF_DATE,,,Mensual,DQ Local,Local,True,True,True,True,True


In [11]:
sdm_componentes['Variables Job'].value_counts()

Variables Job
CUTOFF_DATE                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                              495
CUTOFF_DATE, FECHA_VOLUMETRIC, LOWER_BOUND, UPPER_BOUND                                                                                                                                                                                                                                                                                                      